In [1]:
import os
import openai
import pandas as pd

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
items = pd.read_csv('../data/item_list.txt',sep="\t")

In [4]:
notebook_path = os.path.abspath("2024/Item Zero Shot Learning.ipynb")
print(f"The full path to this notebook is: {notebook_path}")


The full path to this notebook is: /Users/chandler/Library/CloudStorage/Dropbox/teaching/repos/tm-in-class/2024/2024/Item Zero Shot Learning.ipynb


In [4]:
items.sample(10)

,Item
584,Z HOLD Delerium Tremens - L. Hughye
9180,19G Big Little Thing - Sierra Nevada
6897,15P Hand Rolled IPA - Draught Works
2447,21G Breakfast Stout - Founders
5702,13P - String Cheese IPA Oskar Blues
1343,17 Space Dust Pale - Elysian
3104,12M Hofbrau Maibock
2653,27S Delirium Tremens - L. Hughye
1937,16 Bitterroot IPA
7503,26M Key Lime Gose - Überbrew


In [5]:
items.iloc[531]

Item    16 Go To Session IPA - Stone
Name: 531, dtype: object

What do you notice about the items themselves? What do you notice about the names? 

--- 

## Zero Shot Learning

Statistical learning is called "zero shot" when you're just asking a model (in this case, ChatGPT) to carry out a prediction or estimation function with no training examples. For instance, let's extract the beer name from one of our beers, this one with the item name "16 Go To Session IPA - Stone".


In [6]:
system_prompt = """You are a helpful assistant."""

this_beer = items.iloc[531,0]
user_prompt = f"""What beer is in this item name? `{this_beer}`"""

num_tests = 3
responses = []

for _ in range(num_tests) : 

    chat_response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}

            ],
            temperature=0,
            max_tokens=1024,
    )

    responses.append(chat_response.choices[0].message.content)



In [7]:
responses

['The beer in the item name "16 Go To Session IPA - Stone" is "Go To Session IPA" by Stone Brewing.',
 'The beer in the item name "16 Go To Session IPA - Stone" is "Go To Session IPA" by Stone Brewing.',
 'The beer in the item name "16 Go To Session IPA - Stone" is "Go To Session IPA," which is brewed by Stone Brewing.']

We're going to start improving our prompt. 

At this stage we have one main problem. We need the content to come back in a way that we can extract. For instance, if ChatGPT returns 'The beer in the item name `16 Go To Session IPA - Stone` is the Go To Session IPA by Stone Brewing.', it's pretty hard to pull out the beer name. 

Modify the prompt so that you're getting ChatGPT to consistently return the beer name with some sort of delimiter in it to set off the beer name. Feel free to do this in a cell below so that you can compare the results with the above version.


In [ ]:
# Feel free to put your code here



Assuming you've got that working, now we have some new things to consider: 

* Get ChatGPT to extract the beer name with high accuracy. Maybe we should get it to extract the beverage name, since some items aren't beers. 
* Perhaps we want it to handle things like flights or merch or items that aren't a beverage. 

As we work on this prompt, it's useful to know how much money we're spending. The pricing can be found [here](https://openai.com/pricing), though it's not in the easiest form to use. For instance, we're working with GPT 3.5 Turbo. That costs \$0.0015/1K tokens for input and \$0.002 for output. A token is about 4 characters, but we can look at the actual usage in tokens. Running the below, I used 51 tokens (and probably had similar usage on each of the three requests), so I spent about 150/1000 * \$0.0015 = \$0.000225. If I did this 1000 times, it'd cost \$0.225. Cost might matter _some_ here, because there are ultimately 11K items in our list, though that's still just 11 times the big number below. 

In [9]:
chat_response.usage

CompletionUsage(completion_tokens=26, prompt_tokens=34, total_tokens=60, completion_tokens_details={'reasoning_tokens': 0})

In [13]:
total_tokens = chat_response.usage.total_tokens
print(f"Your last test cost {num_tests*total_tokens/1000*0.000150:.6f}.")
print(f"If you did this 1000 times it'd be {num_tests*total_tokens*0.000150:.2f}")

Your last test cost 0.000027.
If you did this 1000 times it'd be 0.03


Open AI also provides a tokenizer tool if you'd like to play around with how many tokens you're using. You can access it [here](https://platform.openai.com/tokenizer).

--- 

Let's now refine our prompts to try to get some better performance. Rather than just looking at this single item, the code below let's you look at a random selection of `num_items`. Refine your prompt until you're satisfied with its performance. 

Once you get to that point, run this for, say, 25 beers, and record the fraction that are perfect. This means that, if a beer is sent in, ChatGPT is extracting exactly the beer name in a way that you could easily pull out of the response. 

In [8]:
system_prompt = """You are a helpful assistant."""

user_prompt_stub = """What beer is in this item name?"""

num_items = 5

random_items = items.sample(num_items)

total_tokens = 0 

for item in random_items.itertuples():
    this_item = item[1]
    user_prompt = user_prompt_stub + f" `{this_item}`"

    chat_response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        max_tokens=1024,
    )

    print("-"*30)
    print(f"The item was {this_item}.")
    print("----- The full is on the next line. -----")
    print(chat_response.choices[0].message.content)

    total_tokens += chat_response.usage.total_tokens

print(f"\n\nThis cost ${num_tests*total_tokens/1000*0.00015:.4f}.")
print(f"If you did this 1000 times it'd be ${num_tests*total_tokens*0.00015:.2f}")

------------------------------
The item was Good Fellow - Pinot Noir.
----- The full is on the next line. -----
The item name "Good Fellow - Pinot Noir" suggests that it is referring to a wine, specifically a Pinot Noir, rather than a beer. Pinot Noir is a type of red wine grape variety known for producing light to medium-bodied wines with soft tannins and a range of flavors. If you are looking for a specific beer, please provide more details or clarify your question!
------------------------------
The item was 33 Pommeau (by the glass) Wandering Aengus.
----- The full is on the next line. -----
The beer in the item name "33 Pommeau (by the glass) Wandering Aengus" is likely "33 Pommeau," which refers to a specific type of beverage. Pommeau is a French aperitif made from a blend of apple juice and apple brandy. However, "Wandering Aengus" is a cider producer based in Oregon, known for their craft ciders. It's possible that "33 Pommeau" is a specific offering from Wandering Aengus, but 